# Unit Commitment Problem with AMPL and Python - Power Grid Lib
[![pglib_uc.ipynb](https://img.shields.io/badge/github-%23121011.svg?logo=github)](https://github.com/ampl/colab.ampl.com/blob/master/authors/nfbvs/pglib_uc/pglib_uc.ipynb) [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ampl/colab.ampl.com/blob/master/authors/nfbvs/pglib_uc/pglib_uc.ipynb) [![Open In Deepnote](https://deepnote.com/buttons/launch-in-deepnote-small.svg)](https://deepnote.com/launch?url=https://github.com/ampl/colab.ampl.com/blob/master/authors/nfbvs/pglib_uc/pglib_uc.ipynb) [![Open In Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/ampl/colab.ampl.com/blob/master/authors/nfbvs/pglib_uc/pglib_uc.ipynb) [![Open In Gradient](https://assets.paperspace.io/img/gradient-badge.svg)](https://console.paperspace.com/github/ampl/colab.ampl.com/blob/master/authors/nfbvs/pglib_uc/pglib_uc.ipynb) [![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/ampl/colab.ampl.com/blob/master/authors/nfbvs/pglib_uc/pglib_uc.ipynb) [![Powered by AMPL](https://h.ampl.com/https://github.com/ampl/colab.ampl.com/blob/master/authors/nfbvs/pglib_uc/pglib_uc.ipynb)](https://ampl.com)

Description: Generic notebook to solve Unit Commitment problems with AMPL and Python using the Power Grid Lib model and test instances.  

Tags: AMPL, amplpy, Python, Power Grid Lib, Unit Commitment Problem, Electric Power Industry

Notebook author: Nicolau Santos <<nicolau@ampl.com>>

In [1]:
# Install dependencies
%pip install -q amplpy pandas

In [2]:
# Google Colab & Kaggle integration
from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["highs", "gurobi"],  # modules to install
    license_uuid="default",  # license to use
)  # instantiate AMPL object and register magics

In [3]:
import json
import time
import pandas as pd
import urllib.request

### Introduction

The Unit Commitment (UC) problem is a mathematical optimization problem in power systems that aims to determine the optimal schedule of power generators to meet electricity demand while minimizing a given objective function subject to various constraints. 

In this notebook we provide an
[AMPL](https://ampl.com/)
model to solve the Unit Commitment problem variant described in 
[Power Grid Lib - Unit Commitment](https://github.com/power-grid-lib/pglib-uc/tree/master) in 
[Python](https://www.python.org/) with 
[amplpy](https://amplpy.ampl.com/en/latest/).


Power Grid Lib - Unit Commitment is a colection of Unit Commitment problems curated and maintained by the [IEEE PES](https://ieee-pes.org/) Task Force on Benchmarks for Validation of Emerging Power System Algorithms.
The benchmark was designed to evaluate a version of the the Unit Commitment problem decribed in [1]. The features of the model are:
- A global load requirement with time series
- An optional global spinning reserve requirement with time series
- Thermal generators with technical parameters, including
    - Minimum and maximum power output
    - Hourly ramp-up and ramp-down rates
    - Start-up and shut-down ramp rates
    - Minimum run-times and off-times
    - Off time dependent start-up costs
    - Piecewise linear convex production costs
    - No-load costs
- Optional renewable generators with time series for minimum and maximum production.

The test instances are divided in three groups where the sets of generators have varying load and reserve profiles: /ca [1], /ferc [2] and /rts_gmlc [3].

In this notebook we provide an AMPL model for the Unit Commitment problem with the above mentioned characteristics. The model is acompanied by a generic function that converts the original *json* data into Python data structures, such as Dictionaries and [Pandas](https://pandas.pydata.org/) Data Frames, and a function that loads the converted data into AMPL with amplpy.

At the end of the notebook example runs with the open source solver [HiGHS](https://highs.dev/) and the [Gurobi](https://www.gurobi.com/) solver are provided.

### Problem description

### Indices and Sets
$g \in \mathcal G$ Set of thermal generators\
$g \in {\mathcal G}_{on}^0$ Set of thermal generators which are initially committed (on)\
$g \in {\mathcal G}_{off}^0$ Set of thermal generators which are not initially committed (off)\
$w \in {\mathcal W}$ set off renewable generators\
$t \in {\mathcal T}$ Hourly time steps: $1..T, T=time\_periods$\
$l \in {\mathcal L}_g$ Piecewise production cost intervals for thermal generator $g: 1..L_g$.\
$s \in {\mathcal S}_g$ Startup categories for thermal generator $g$, from *hottest*(1) to *coldest* ($S_g$): $1..S_g$.

### System Parameters
$D(t)$ Load (demand) at time $t$ (MW), `demand`\
$R(t)$ Spinning reserve at time $t$ (MW), `reserves`

### Thermal Generator Parameters
$CS_g^s$ Startup cost in category $s$ for generator $g$ ().\
$CP_g^l$ Cost of operating at piecewise generation point $l$ for generator $g$ (MW).\
$DT_g$ Minimum downtime for generator $g$ (h), timedown minimum.\
$DT_g^0$ Number of time periods the unit has been off prior to the first time period for generator $g$, timedownt0.\
$\overline{P}_g$ Maximum power output for generator $g$ (MW), poweroutput maximum.\
$\underline{P}_g$ Minimum power output for generator $g$ (MW), poweroutput minimum.\
$P_g^0$ Power output for generator $g$ (MW) in the time period prior to $t=1$, `poweroutputt0`\
$P_g^l$ Power level for piecewise generation point $l$ for generator $g$ (MW); $P_g^1=P_g$ andPLg g =Pg,`piecewiseproduction[mw ]`\
$RD_g$ Ramp-down rate for generator $g$ (MW/h), `rampdownlimit`.\
$RU_g$ Ramp-up rate for generator $g$ (MW/h), `rampuplimit`.\
$SD_g$ Shutdown capability for generator $g$ (MW), `rampshutdownlimit`\
$SU_g$ Startup capability for generator $g$ (MW), `rampstartuplimit`\
$TS_g^s$ Time offline after which the startup category $s$ becomes active(h), `startup[lag]`.\
$UT_g$ Minimum uptime for generator $g$ (h), `timeupminimum`.\
$UT_g^0$ Number of time periods the unit has been on prior to the first time period for generator $g$, `timeupt0`.\
$U_g^0$ Initial on/off status for generator $g$, $U_g^0=1$ for $g \in \mathcal{G}_{on}^0$, $U_g^0=0$ for $g \in \mathcal{G}_{off}^0$ `unitont0`.\
$U_g$ Must-run status for generator $g$, `mustrun`.

$$
\text{min } \sum_{g \in {\mathcal G}} \sum_{t \in {\mathcal T}} \left( c_g(t) + CP_g^1 \, u_g(t) + \sum_{s = 1}^{S_g} \left( CS^s_g \delta^s(t) \right) \right) \hspace{1cm} (1)
$$



$$
\begin{split}
\text{subject to:}\\
%\label{eq:UCDemand}
& \sum_{g \in {\mathcal G}} \left( p_g(t) + \underline{P}_g u_g(t) \right) + \sum_{w\in {\mathcal W}} p_w(t) = D(t) & \hspace{5cm} \forall t \in {\mathcal T} & \hspace{1cm} (2) \\
%\label{eq:UCReserves}
& \sum_{g \in {\mathcal G}} r_g(t) \geq R(t) &  \forall t \in {\mathcal T} & \hspace{1cm} (3) \\
%\label{eq:initialUpRequirement}
& \sum_{t=1}^{\min\{UT_g - UT_g^0, T\}} (u_g(t) - 1) = 0 & \hspace{3cm} \forall g \in {\mathcal G}_{\textit{on}}^0 & \hspace{1cm} (4) \\
% \label{eq:initialDownRequirement}
& \sum_{t=1}^{\min\{DT_g - DT_g^0, T\}} u_g(t) = 0 & \forall g \in {\mathcal G}_{\textit{off}}^0 & \hspace{1cm} (5) \\
% \label{eq:LogicalInitial}
& u_g(1) - U_g^0 = v_g(1) - w_g(1) & \forall g \in {\mathcal G} & \hspace{1cm} (6) \\
%  \label{eq:STIInit}
& \sum_{s=1}^{S_g-1} \sum_{t=\max\{1, TS^{s+1}_g - DT^0_g + 1\}}^{\min\{TS^{s+1}_g -1,T\}} \delta^s_g(t) = 0 & \forall g \in {\mathcal G}& \hspace{1cm} (7) \\
% \label{eq:RampUpInit}
& p_g(1) + r_g(1) - U_g^0(P_g^0-\underline{P}_g) \leq RU_g & \forall g \in {\mathcal G} & \hspace{1cm} (8) \\
% \label{eq:RampDownInit}
& U_g^0(P_g^0-\underline{P}_g) - p_g(1) \leq RD_g & \forall g \in {\mathcal G} & \hspace{1cm} (9) \\
% \label{eq:MaxOutput2Init}
& U_g^0(P_g^0-\underline{P}_g) \leq (\overline{P}_g - \underline{P}_g) U_g^0 - \max\{(\overline{P}_g - SD_g),0\} w_g(1) & \forall g \in {\mathcal G} & \hspace{1cm} (10) \\
% \label{eq:MustRun}
& u_g(t) \geq U_g & \hspace{1cm} \forall t \in {\mathcal T}, \, \forall g \in {\mathcal G} & \hspace{1cm} (11) \\
% \label{eq:Logical}
& u_g(t) - u_g(t-1) = v_g(t) - w_g(t) & \forall t \in {\mathcal T}\setminus\{1\}, \, \forall g \in {\mathcal G} & \hspace{1cm} (12) \\
% \label{eq:Startup}
& \sum_{i= t-\min\{UT_g,T\} + 1}^t v_g(i) \leq u_g(t) & \forall t \in \{\min\{UT_g,T\} \ldots, T\}, \, \forall g \in {\mathcal G} & \hspace{1cm} (13) \\
% \label{eq:Shutdown}
& \sum_{i= t-\min\{DT_g,T\} + 1}^t w_g(i) \leq 1 - u_g(t) & \forall t \in \{\min\{DT_g, T\}, \ldots, T\}, \, \forall g \in {\mathcal G} & \hspace{1cm} (14) \\
% \label{eq:STISelect}
& \delta^s_g(t) \leq \sum_{i = TS^s_g}^{TS^{s+1}_g-1} w_g(t-i) & \forall t \in \{TS^{s+1}_g,\ldots,T\},\,\forall s \in {\mathcal T}_g\!\setminus\!\{S_g\},\,  \forall g \in {\mathcal G} & \hspace{1cm} (15) \\
% \label{eq:STILink}
& v_g(t) = \sum_{s = 1}^{S_g} \delta^s_g(t) & \forall t \in {\mathcal T},\, \forall g \in {\mathcal G} & \hspace{1cm} (16) \\
% \label{eq:MaxOutput1}
& p_g(t) + r_g(t) \leq (\overline{P}_g - \underline{P}_g) u_g(t) - \max\{(\overline{P}_g - SU_g),0\} v_g(t) & \forall t \in {\mathcal T}, \, \forall g \in {\mathcal G} & \hspace{1cm} (17) \\
%\label{eq:MaxOutput2}
& p_g(t) + r_g(t) \leq (\overline{P}_g - \underline{P}_g) u_g(t) - \max\{(\overline{P}_g - SD_g),0\} w_g(t+1) & \forall t \in {\mathcal T}\setminus \{T\}, \,\forall g \in {\mathcal G} & \hspace{1cm} (18) \\
% \label{eq:RampUp}
& p_g(t) + r_g(t) - p_g(t-1) \leq RU_g & \forall t \in {\mathcal T}\setminus\{1\}, \, \forall g \in {\mathcal G} & \hspace{1cm} (19) \\
% \label{eq:RampDown}
& p_g(t-1) - p_g(t) \leq RD_g & \forall t \in {\mathcal T}\setminus\{1\}, \, \forall g \in {\mathcal G} & \hspace{1cm} (20) \\
% \label{eq:PiecewiseParts} & \hspace{1cm} (2) \\
& p_g(t) = \sum_{l \in {\mathcal L}_g} (P_g^l - P_g^1) \lambda_g^l(t) &\hspace{5cm} \forall t \in {\mathcal T}, \, \forall g \in {\mathcal G} & \hspace{1cm} (21) \\
% \label{eq:PiecewisePartsCost}
& c_g(t) = \sum_{l \in {\mathcal L}_g} (CP_g^l - CP_g^1) \lambda_g^l(t) & \forall t \in {\mathcal T}, \, \forall g \in {\mathcal G} & \hspace{1cm} (22) \\
% \label{eq:PiecewiseLimits}
& u_g(t) = \sum_{l \in {\mathcal L}_g} \lambda_g^l(t) & \forall t \in {\mathcal T}, \forall g \in {\mathcal G} & \hspace{1cm} (23) \\
% \label{eq:WindLimit}
& \underline{P}_w(t) \leq p_w(t) \leq \overline{P}_w(t) &\hspace{6cm} \forall t \in {\mathcal T}, \, \forall w \in {\mathcal W} & \hspace{1cm} (24) 
\end{split}
$$

### AMPL model

In [4]:
%%writefile uc.mod

set thermal_gens;
set renewable_gens;

param S {thermal_gens};
set gen_startup_categories {g in thermal_gens} := 1..S[g];

param startup_lag  {g in thermal_gens, gen_startup_categories[g]};
param startup_cost {g in thermal_gens, gen_startup_categories[g]};

param L {thermal_gens};
set gen_pwl_points {g in thermal_gens} := 1..L[g];

param piecewise_mw   {g in thermal_gens, gen_pwl_points[g]};
param piecewise_cost {g in thermal_gens, gen_pwl_points[g]};

param T;
set time_periods := 1..T;

param demand   {time_periods};
param reserves {time_periods};

param must_run             {thermal_gens};
param power_output_minimum {thermal_gens};
param power_output_maximum {thermal_gens};
param ramp_up_limit        {thermal_gens};
param ramp_down_limit      {thermal_gens};
param ramp_startup_limit   {thermal_gens};
param ramp_shutdown_limit  {thermal_gens};
param time_up_minimum      {thermal_gens};
param time_down_minimum    {thermal_gens};
param power_output_t0      {thermal_gens};
param unit_on_t0           {thermal_gens};
param time_down_t0         {thermal_gens};
param time_up_t0           {thermal_gens};


# Renewable Generator Parameters
param ren_power_output_minimum {renewable_gens, time_periods};
param ren_power_output_maximum {renewable_gens, time_periods};

# Variables
var cg {thermal_gens, time_periods};
var pg {thermal_gens, time_periods} >= 0;
var rg {thermal_gens, time_periods} >= 0;
var pw {renewable_gens, time_periods} >= 0;
var ug {thermal_gens, time_periods} binary;
var vg {thermal_gens, time_periods} binary;
var wg {thermal_gens, time_periods} binary;
var dg {g in thermal_gens, gen_startup_categories[g], time_periods} binary;
var lg {g in thermal_gens, gen_pwl_points[g], time_periods} >= 0, <= 1;

# Objective

#(1)
minimize obj:
	sum{g in thermal_gens, t in time_periods}(
		cg[g,t] + 
		piecewise_cost[g, 1] * ug[g,t] + 
		sum{s in gen_startup_categories[g]}(
			startup_cost[g, s] * dg[g,s,t]
		)
	);

# Constraints

#(2)
s.t. UCDemand {t in time_periods}:
	sum{g in thermal_gens}(pg[g,t] + power_output_minimum[g] * ug[g,t]) + sum{w in renewable_gens} pw[w,t] == demand[t];

#(3)
s.t. UCReserves {t in time_periods}:
	sum{g in thermal_gens} rg[g,t] >= reserves[t];

#(4)
s.t. initialUpRequirement {g in thermal_gens: unit_on_t0[g] == 1}:
	sum{t in 1 .. min(time_up_minimum[g] - time_up_t0[g], T)} (ug[g,t] - 1) == 0;

#(5)
s.t. initialDownRequirement {g in thermal_gens: unit_on_t0[g] == 0}:
	sum{t in 1 .. min(time_down_minimum[g] - time_down_t0[g], T)} ug[g,t] == 0;

#(6)
s.t. LogicalInitial {g in thermal_gens}:
	ug[g,1] - unit_on_t0[g] == vg[g,1] - wg[g,1];

#(7)
s.t. STIInit {g in thermal_gens}:
	sum{
		s in 1..(S[g]-1),
		t in
			(max(1, startup_lag[g, s+1] - time_down_t0[g] + 1)) ..
			(min(startup_lag[g, s+1]-1, T))
	} dg[g,s,t] == 0;

#(8)
s.t. RampUpInit {g in thermal_gens}:
	pg[g,1] + rg[g,1] - unit_on_t0[g] * (power_output_t0[g] - power_output_minimum[g]) <= ramp_up_limit[g];

#(9)
s.t. RampDownInit {g in thermal_gens}:
	unit_on_t0[g] * (power_output_t0[g] - power_output_minimum[g]) - pg[g,1] <= ramp_down_limit[g];

#(10)
s.t. MaxOutput2Init {g in thermal_gens}:
	unit_on_t0[g] * (power_output_t0[g] - power_output_minimum[g]) <=
		unit_on_t0[g] *(power_output_maximum[g] - power_output_minimum[g]) - max((power_output_maximum[g] - ramp_shutdown_limit[g]),0) * wg[g,1];

#(11)
s.t. MustRun {g in thermal_gens, t in time_periods}:
	ug[g,t] >= must_run[g];

#(12)
s.t. Logical {g in thermal_gens, t in time_periods: t != 1}:
	ug[g,t] - ug[g,t-1] == vg[g,t] - wg[g,t];

#(13)
s.t. Startup {g in thermal_gens, t in min(time_up_minimum[g], T) .. T}:
	sum{i in (t - min(time_up_minimum[g], T) + 1).. t} vg[g,i] <= ug[g,t];

#(14)
s.t. Shutdown {g in thermal_gens, t in min(time_down_minimum[g], T) .. T}:
	sum{i in (t - min(time_down_minimum[g], T) + 1) .. t} wg[g,i] <= 1 - ug[g,t];

#(15)
s.t. STISelect {
	g in thermal_gens,
	s in gen_startup_categories[g],
	t in startup_lag[g, s+1] .. T:
		s != S[g]
	}:
	dg[g,s,t] <= sum{i in startup_lag[g, s] .. (startup_lag[g, s+1]-1)} wg[g,t-i];

#(16)
s.t. STILink {g in thermal_gens, t in time_periods}:
	vg[g,t] == sum{s in 1..S[g]} dg[g,s,t];

#(17)
s.t. MaxOutput1 {g in thermal_gens, t in time_periods}:
	pg[g,t] + rg[g,t] <=
	(power_output_maximum[g] - power_output_minimum[g]) * ug[g,t] - 
	max((power_output_maximum[g] - ramp_startup_limit[g]),0) * vg[g,t];

#(18)
s.t. MaxOutput2 {g in thermal_gens, t in time_periods: t != T}:
	pg[g,t] + rg[g,t] <=
	(power_output_maximum[g] - power_output_minimum[g]) * ug[g,t] - 
	max((power_output_maximum[g] - ramp_shutdown_limit[g]),0) * wg[g,t+1];

#(19)
s.t. RampUp {g in thermal_gens, t in time_periods: t != 1}:
	pg[g,t] + rg[g,t] - pg[g,t-1] <= ramp_up_limit[g];

#(20)
s.t. RampDown {g in thermal_gens, t in time_periods: t != 1}:
	pg[g,t-1] - pg[g,t] <= ramp_down_limit[g];

#(21)
s.t. PiecewiseParts {g in thermal_gens, t in time_periods}:
	pg[g,t] == sum{l in gen_pwl_points[g]}(piecewise_mw[g,l] - piecewise_mw[g,1]) * lg[g,l,t];

#(22)
s.t. PiecewisePartsCost {g in thermal_gens, t in time_periods}:
	cg[g,t] == sum{l in gen_pwl_points[g]}((piecewise_cost[g,l] - piecewise_cost[g,1]) * lg[g,l,t]);

#(23)
s.t. PiecewiseLimits {g in thermal_gens, t in time_periods}:
	ug[g,t] == sum{l in gen_pwl_points[g]} lg[g,l,t];

#(24)
s.t. WindLimit {w in renewable_gens, t in time_periods}:
	ren_power_output_minimum[w,t] <= pw[w,t] <= ren_power_output_maximum[w,t];

### Data preparation

In [5]:
def prepare_pglib_uc(data_file, log=True):

    data = json.load(open(data_file, "r"))

    thermal_gens_data = data["thermal_generators"]
    renewable_gens_data = data["renewable_generators"]

    startup_info = []
    piecewise_production_info = []

    T = data["time_periods"]
    S = {}
    L = {}

    for k, v in thermal_gens_data.items():

        for i, val in enumerate(v["startup"]):
            startup_info.append([k, i + 1, val["lag"], val["cost"]])

        S[k] = len(v["startup"])

        for i, val in enumerate(v["piecewise_production"]):
            piecewise_production_info.append([k, i + 1, val["mw"], val["cost"]])

        L[k] = len(v["piecewise_production"])

        del v["startup"]
        del v["piecewise_production"]

    df_thermal_gens = pd.DataFrame(thermal_gens_data).transpose()
    df_thermal_gens = df_thermal_gens.drop("name", axis=1)

    df_startup = pd.DataFrame(
        startup_info, columns=["gen", "cat", "startup_lag", "startup_cost"]
    ).set_index(["gen", "cat"])

    df_piecewise_production = pd.DataFrame(
        piecewise_production_info,
        columns=["gen", "int", "piecewise_mw", "piecewise_cost"],
    ).set_index(["gen", "int"])

    renewable_gens = list(renewable_gens_data)

    ren_power_output_minimum = {}
    ren_power_output_maximum = {}

    for k, v in renewable_gens_data.items():

        p_min = v["power_output_minimum"]

        for i, val in enumerate(p_min):
            ren_power_output_minimum[(k, i + 1)] = val

        p_max = v["power_output_maximum"]

        for i, val in enumerate(p_max):
            ren_power_output_maximum[(k, i + 1)] = val

    demand = data["demand"]
    reserves = data["reserves"]

    # pack everything in a dict and return data
    ampl_data = {}
    ampl_data["T"] = T
    ampl_data["S"] = S
    ampl_data["L"] = L
    ampl_data["demand"] = demand
    ampl_data["reserves"] = reserves
    ampl_data["renewable_gens"] = renewable_gens
    ampl_data["ren_power_output_minimum"] = ren_power_output_minimum
    ampl_data["ren_power_output_maximum"] = ren_power_output_maximum
    ampl_data["df_thermal_gens"] = df_thermal_gens
    ampl_data["df_startup"] = df_startup
    ampl_data["df_piecewise_production"] = df_piecewise_production

    return ampl_data

### Function wrapper

In [6]:
def run_uc(data, solver="gurobi", solver_options=None, log=True):

    start_time = time.time()

    if log:
        print("Starting run_uc")

    # instantiate AMPL and load model
    ampl = AMPL()
    ampl.read("uc.mod")

    # load data
    if log:
        print("Loading data")

    ampl.set_data(data["df_thermal_gens"], "thermal_gens")
    ampl.param["S"] = data["S"]
    ampl.param["L"] = data["L"]
    ampl.param["T"] = data["T"]

    ampl.set["renewable_gens"] = data["renewable_gens"]
    ampl.param["ren_power_output_minimum"] = data["ren_power_output_minimum"]
    ampl.param["ren_power_output_maximum"] = data["ren_power_output_maximum"]

    ampl.set_data(data["df_startup"])
    ampl.set_data(data["df_piecewise_production"])

    ampl.param["demand"] = data["demand"]
    ampl.param["reserves"] = data["reserves"]

    # set solver and options
    if log:
        print("Setting solver and options")

    ampl.option["solver"] = solver

    if solver_options is not None:
        ampl.option[solver + "_options"] = solver_options

    # solve
    if log:
        print("Solving")
        ampl.solve()
    else:
        ampl.get_output("solve;")

    # check solve result and time
    solve_result = ampl.get_value("solve_result")
    solve_time = ampl.get_value("_total_solve_elapsed_time")

    assert ampl.solve_result in ["solved", "limit"], ampl.solve_result

    if solve_result != "solved":
        print("WARNING: solver returned '%s' status" % (solve_result,))

    # get result info
    # objective
    objective = ampl.obj["obj"].value()
    # dataframe with variables indexed by thermal_gens and time_periods
    df_tg_tp = ampl.get_data("cg", "pg", "rg", "ug", "vg", "wg").to_pandas()
    # dataframe with variables indexed by renewable_gens and time_periods
    df_rg_tp = ampl.get_data("pw").to_pandas()
    # dataframe with variables indexed by renewable_gens, gen_startup_categories and time_periods
    df_dg = ampl.get_data("dg").to_pandas()
    # dataframe with variables indexed by renewable_gens, gen_pwl_points and time_periods
    df_lg = ampl.get_data("lg").to_pandas()

    var_dict = {
        "thermal_info": df_tg_tp,
        "renewable_info": df_rg_tp,
        "dg_df": df_dg,
        "lg_df": df_lg,
    }

    end_time = time.time()

    result = {
        "nvars": ampl.get_value("_nvars"),
        "ncons": ampl.get_value("_ncons"),
        "objective": objective,
        "solve_result": solve_result,
        "solve_time": solve_time,
        "total_time": end_time - start_time,
        "vars": var_dict,
    }

    return result

### Numerical example

In [7]:
# download sample instance
url = "https://raw.githubusercontent.com/power-grid-lib/pglib-uc/refs/heads/master/rts_gmlc/2020-02-09.json"
file = "2020-02-09.json"

urllib.request.urlretrieve(url, file)
data = prepare_pglib_uc(file)

### Solve with HiGHS

In [8]:
result_highs = run_uc(
    data, solver="highs", solver_options="outlev=1 timelim=30 threads=16"
)
print("objective:", result_highs["objective"])
assert result_highs["solve_result"] in ["solved", "limit"], ampl.solve_result

Starting run_uc
Loading data
Setting solver and options
Solving
HiGHS 1.8.1:   tech:outlev = 1
  lim:time = 30
  tech:threads = 16
Running HiGHS 1.7.1 (git hash: 43329e5): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 5e+03]
  Cost   [1e+00, 4e+04]
  Bound  [1e-01, 2e+05]
  RHS    [1e+00, 4e+03]
Presolving model
28526 rows, 34765 cols, 131950 nonzeros  0s
23661 rows, 29890 cols, 155559 nonzeros  0s
22404 rows, 27425 cols, 157604 nonzeros  0s

Solving MIP model with:
   22404 rows
   27425 cols (10102 binary, 0 integer, 0 implied int., 17323 continuous)
   157604 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   20304.91        inf                  inf        0      0      0         0     1.0s
         0    

### Solve with Gurobi

In [9]:
result_gurobi = run_uc(data, solver="gurobi", solver_options="outlev=1")
print("objective:", result_gurobi["objective"])
assert result_gurobi["solve_result"] in ["solved", "limit"], ampl.solve_result

Starting run_uc
Loading data
Setting solver and options
Solving
Gurobi 12.0.0: Set parameter LogToConsole to value 1
  tech:outlev = 1
Set parameter InfUnbdInfo to value 1
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
InfUnbdInfo  1

Optimize a model with 34617 rows, 40986 columns and 151682 nonzeros
Model fingerprint: 0x75bda483
Variable types: 25197 continuous, 15789 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 4e+04]
  Bounds range     [1e-01, 2e+05]
  RHS range        [1e+00, 4e+03]
Presolve removed 7614 rows and 12523 columns
Presolve time: 0.30s
Presolved: 27003 rows, 28463 columns, 123106 nonzeros
Variable types: 17188 continuous, 11275 integer (11275 binary)
Found heuristic solution: ob

### References

[1]
Knueven, Bernard and Ostrowski, James and Watson, Jean-Paul.
On mixed integer programming formulations for the unit commitment problem.
INFORMS Journal on Computing
(2020).

[2]
Krall, Eric and Higgins, Michael and O’Neill, Richard P.
RTO unit commitment test system.
Federal Energy Regulatory Commission
(2012).

[3]
Barrows, Clayton, Aaron Bloom, Ali Ehlen, Jussi Ikaheimo, Jennie Jorgenson, Dheepak Krishnamurthy, Jessica Lau et al.
The IEEE Reliability Test System: A Proposed 2019 Update.
IEEE Transactions on Power Systems
(2019).

[4]
Morales-España, Germán and Latorre, Jesus M and Ramos, Andres.
Tight and compact MILP formulation for the thermal unit commitment problem.
IEEE Transactions on Power Systems
(2013).

[5] 
Sridhar, Srikrishna and Linderoth, Jeff and Luedtke, James
Locally ideal formulations for piecewise linear functions with indicator variables.
Operations Research Letters
(2013).